<a href="https://colab.research.google.com/github/JinnyKo/LLaMA2_PretrainTest/blob/master/Llama2_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
from datasets import load_dataset, DatasetDict

raw_dataset = load_dataset('ccdv/cnn_dailymail', '3.0.0')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for ccdv/cnn_dailymail contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/cnn_dailymail
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [90]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [91]:
raw_dataset['train'][0]['article'][:200]

"It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"

In [92]:
raw_dataset['train'].to_pandas()

,article,highlights,id
0,It's official: U.S. President Barack Obama wan...,Syrian official: Obama climbed to the top of t...,0001d1afc246a7964130f43ae940af6bc6c57f01
1,(CNN) -- Usain Bolt rounded off the world cham...,Usain Bolt wins third gold of world championsh...,0002095e55fcbd3a2f366d9bf92a95433dc305ef
2,"Kansas City, Missouri (CNN) -- The General Ser...",The employee in agency's Kansas City office is...,00027e965c8264c35cc1bc55556db388da82b07f
3,Los Angeles (CNN) -- A medical doctor in Vanco...,NEW: A Canadian doctor says she was part of a ...,0002c17436637c4fe1837c935c04de47adb18e9a
4,(CNN) -- Police arrested another teen Thursday...,Another arrest made in gang rape outside Calif...,0003ad6ef0c37534f80b55b4235108024b407f0b
...,...,...,...
287108,Tiger Woods’s frustration at the lamentable st...,"Woods said: ’Guys, give me a little f***ing sp...",fffdfb56fdf1a12d364562cc2b9b1d4de7481dee
287109,By . Mark Duell . Last updated at 4:07 PM on 2...,13 sailors died in 1804 after explosives ship ...,fffeecb8690b85de8c3faed80adbc7a978f9ae2a
287110,"Suicide: Troll victim Hannah Smith, 14, killed...",Hannah Smith's father says Ask.fm's safety cha...,ffff5231e4c71544bc6c97015cdb16c60e42b3f4
287111,By . Victoria Woollaston and Mark Prigg . PUBL...,A test version of Windows 8.1 is available to ...,ffff924b14a8d82058b6c1c5368ff1113c1632af


## 모델 학습 코드 만들때, 데이터 다 넣지 말고 작게 나눠서 test

In [93]:
sample_dataset = DatasetDict(
    {
        "train": raw_dataset['train'].select(range(50000)).shuffle(),
        "valid": raw_dataset['test'].select(range(5000)).shuffle()
    }
)

## tokenizer: GPT2 tokenizer 사용
### => Model train 할 떄랑 다르게 tokenizer 쓸 때는 원본 데이터 다 써도 그렇게 느리지 않은 듯

In [94]:
from transformers import AutoTokenizer

In [95]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [96]:
def get_training_corpus(ds):
  return(
      #1000개씩 나눠서
      ds[i:i+1000]['article'] for i in range(0,len(ds), 1000)
  )

  training_corpus = get_training_corpus(raw_dataset['train'])

In [97]:
#시간 확인
%%time
training_corpus = get_training_corpus(raw_dataset['train'])
tokenizer = tokenizer.train_new_from_iterator(training_corpus, vocab_size = 50257)

CPU times: user 11min 48s, sys: 7.02 s, total: 11min 55s
Wall time: 1min 42s


In [98]:
sample_text = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours afte"
tokenizer.tokenize(sample_text)

['It',
 "'s",
 'Ġofficial',
 ':',
 'ĠU',
 '.',
 'S',
 '.',
 'ĠPresident',
 'ĠBarack',
 'ĠObama',
 'Ġwants',
 'Ġlawmakers',
 'Ġto',
 'Ġweigh',
 'Ġin',
 'Ġon',
 'Ġwhether',
 'Ġto',
 'Ġuse',
 'Ġmilitary',
 'Ġforce',
 'Ġin',
 'ĠSyria',
 '.',
 'ĠObama',
 'Ġsent',
 'Ġa',
 'Ġletter',
 'Ġto',
 'Ġthe',
 'Ġheads',
 'Ġof',
 'Ġthe',
 'ĠHouse',
 'Ġand',
 'ĠSenate',
 'Ġon',
 'ĠSaturday',
 'Ġnight',
 ',',
 'Ġhours',
 'Ġaf',
 'te']

## Tokenize 사용하지 않고 바로 넣어주면??
### - token 대신에 input_id 랑 attention_mask 나옴


In [99]:
tokenizer(sample_text, return_length=True)

{'input_ids': [868, 345, 1061, 26, 458, 14, 51, 14, 1497, 4149, 1288, 2880, 8505, 280, 6284, 285, 316, 1714, 280, 1321, 1681, 2692, 285, 2730, 14, 1288, 1170, 259, 3102, 280, 262, 6017, 290, 262, 1821, 292, 3897, 316, 1722, 1074, 12, 1511, 454, 865], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'length': [44]}

In [77]:
#20240329 issue: TypeError: tokenizer() got an unexpected keyword argument 'max_lengthm, // Hugging Face transformers tokenizer =>  max_length 인자로 사용 하는지 확인하기.

#모델에 128 길이 context
#context_length = 128

In [122]:
def tokenize(batch):
    outputs = tokenizer (
        batch['article'],
        truncation=True,
        return_overflowing_tokens = True,
        return_length=True
    )

    input_batch = []
    for length, input_ids in zip(outputs['length'],outputs['input_ids']):
        if length == context_length:
          input_batch.append(input_ids)
    return {"input_ids": input_batch}


In [124]:
# sample_dataset이 딕셔너리로 가정되었으므로 키에 직접 접근하여 데이터셋을 가져오게 수정
tokenized_datasets = sample_dataset.map(tokenize, batched=True, remove_columns=sample_dataset['train'].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

## Load_model


In [125]:
from transformers import LlamaConfig

configuration = LlamaConfig()
configuration

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [126]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.vocab_size

(0, 0, 50257)

In [127]:
#tokenizer 값 맞춰주기
#전반적으로 사이즈 기본 값에서 8배정도 줄임
configuration = LlamaConfig(**{
  "attention_bias": False,
  "attention_dropout": 0.0,
  "bos_token_id": 0, #Jinny 기본값에서 내 Tokenizer맞게 수정
  "eos_token_id": 0, #Jinny 기본값에서 수정
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1376,
  "max_position_embeddings": 128,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": None, #20240329 error
  "rope_theta": 10000.0,
  "tie_word_embeddings": False,
  "transformers_version": "4.38.2",
  "use_cache": True,
  "vocab_size": 50257 #Jinny 기본값에서 내 Tokenizer맞게 수정
})

In [128]:
from transformers import LlamaForCausalLM
model = LlamaForCausalLM(configuration)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=4096, bias=False)
          (v_proj): Linear(in_features=512, out_features=4096, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_feat

In [129]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [130]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=4096, bias=False)
          (v_proj): Linear(in_features=512, out_features=4096, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Linear(in_feat

In [131]:
#학습 전 Test => 국가 이름 가렸음
prompt = "It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in "
inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=50)
generate_ids

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


RuntimeError: The size of tensor a (4) must match the size of tensor b (0) at non-singleton dimension 1